In [1]:
import sys
import os

# Get absolute path to your project root (one level above eeg_pretraining)
project_root = os.path.abspath(os.path.join(os.getcwd()))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from eeg_pretraining.pretrainer_model import EEGPretrainer
from eeg_text_dataset import EEGTextDataset, collate_fn

In [3]:
# === Config ===
batch_size = 16
epochs = 20
lr = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Data ===
dataset = EEGTextDataset("BCI_trainingData/")
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# === Model ===
model = EEGPretrainer().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

[resultsYDG_NR_pairs.mat] Index 0: word = 'ford', eeg shape = (105, 98)
[resultsYDG_NR_pairs.mat] Index 1: word = 'edsel', eeg shape = (105, 140)
[resultsYDG_NR_pairs.mat] Index 2: word = 'founded', eeg shape = (105, 94)
[resultsYDG_NR_pairs.mat] Index 3: word = 'foundation', eeg shape = (105, 77)
[resultsYDG_NR_pairs.mat] Index 4: word = '1936', eeg shape = (105, 169)
[resultsYDG_NR_pairs.mat] Index 5: word = 'a', eeg shape = (105, 76)
[resultsYDG_NR_pairs.mat] Index 6: word = 'local', eeg shape = (105, 115)
[resultsYDG_NR_pairs.mat] Index 7: word = 'philanthropic', eeg shape = (105, 84)
[resultsYDG_NR_pairs.mat] Index 8: word = 'organization', eeg shape = (105, 151)
[resultsYDG_NR_pairs.mat] Index 9: word = 'broad', eeg shape = (105, 158)
[resultsYDG_NR_pairs.mat] Index 10: word = 'charter', eeg shape = (105, 103)
[resultsYDG_NR_pairs.mat] Index 11: word = 'to', eeg shape = (105, 111)
[resultsYDG_NR_pairs.mat] Index 12: word = 'promote', eeg shape = (105, 109)
[resultsYDG_NR_pairs.ma

In [4]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        eeg = batch['eeg'].to(device)  # [B, T, C]

        pred, mask, target = model(eeg)  # unpatched prediction and mask
        loss = criterion(pred[mask], target[mask])  # MSE only on valid positions

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")


DEBUG: <class 'numpy.ndarray'> [[-6.65174358 -3.44467972 -3.29610664 ... -4.89419179 -2.49511667
   0.        ]
 [-6.84282793 -4.29662349 -4.35251415 ... -7.08134374 -4.04340648
   0.        ]
 [-6.12566058 -4.41915931 -4.62138528 ... -8.1595117  -4.96369842
   0.        ]
 ...
 [ 4.84000361  1.72805808  1.37037044 ...  2.33529039  0.97350263
   0.        ]
 [ 6.1125109   2.51606194  2.43877035 ...  3.65799602  1.86784093
   0.        ]
 [ 6.58394864  2.93179187  3.12120228 ...  4.38941356  2.43053016
   0.        ]]
DEBUG: <class 'numpy.ndarray'> [[ 4.27255724 -0.09448058  2.46623024 ... -1.65907458  2.23355492
   0.        ]
 [ 4.49390953  0.19575982  1.96753976 ... -1.40997045  2.25846633
   0.        ]
 [ 4.60232334  0.57869051  1.40556342 ... -1.00768465  2.22075121
   0.        ]
 ...
 [ 4.5417187   8.25883271  8.66059866 ...  9.42580646  9.67827892
   0.        ]
 [ 5.46083201  9.13531942  9.14371913 ...  9.93592918 10.58251959
   0.        ]
 [ 6.13234051  9.46204528  9.2738311

RuntimeError: The size of tensor a (25) must match the size of tensor b (16) at non-singleton dimension 1

In [ ]:
torch.save(model.state_dict(), "eeg_pretrained.pth")
